In [15]:
import psycopg2
import pandas as pd
import csv
import re
from sklearn.metrics import r2_score

In [2]:
config = {
    "user": "postgres",
    "password": "admin",
    "database": "etl_db"
}

def create_connection():
    try:
        cnx = psycopg2.connect(
            host='localhost',
            user=config["user"],
            password=config["password"],
            database=config["database"]
        )
        print('Conexión exitosa!!')
    except psycopg2.Error as e:
        cnx = None
        print('No se puede conectar:', e)
    return cnx

In [5]:
create_connection()

Conexión exitosa!!


<connection object at 0x000002694DF089D0; dsn: 'user=postgres password=xxx dbname=etl_db host=localhost', closed: 0>

In [7]:
creacion =""" CREATE TABLE modelo (
    Country VARCHAR(255),
    Happiness_Score FLOAT,
    GDP FLOAT,
    Family FLOAT,
    Life_Expectancy FLOAT,
    Freedom FLOAT,
    Corruption FLOAT,
    Generosity FLOAT,
    prediccion FLOAT
    
);
"""
try:
    conexion = create_connection()
    cursor = conexion.cursor()
    
    cursor.execute(creacion)
    conexion.commit()
    print("Tabla creada exitosamente.")
    
except psycopg2.Error as e:
    print("Error al crear la tabla:", e)
    
finally:
    if conexion:
        conexion.close()

Conexión exitosa!!
Tabla creada exitosamente.


In [27]:

def traer_tabla():
    try:
        conexion = create_connection()
        cursor = conexion.cursor()
        
    
        # Realiza una consulta SQL para obtener todos los datos de la tabla
        query = "SELECT * FROM modelo"
        cursor.execute(query)
    
        # Obtén todos los resultados como una lista de tuplas
        data = cursor.fetchall()
        #print(data)
    
        column_names = [desc[0] for desc in cursor.description]
    
        # Crea un DataFrame de pandas con los datos y nombres de columnas
        df = pd.DataFrame(data, columns=column_names)
        return df
    except psycopg2.Error as e:
        print("Error al crear la tabla:", e)
        
    finally:
        if conexion:
            conexion.close()




# Llama a la función para obtener los datos desde la base de datos
base_datos= traer_tabla()
base_datos=base_datos.drop_duplicates()

print(base_datos)
# Supongamos que 'y_true' son los valores reales y 'y_pred' son las predicciones
real = base_datos['happiness_score']
prediccion = base_datos['prediccion']

r_squared = r2_score(real, prediccion)

print("el r cuadrado es", r_squared)

Conexión exitosa!!
           country  happiness_score      gdp   family  life_expectancy  \
0      Switzerland            7.587  1.39651  1.34951          0.94143   
1           Norway            7.522  1.45900  1.33095          0.88521   
2      New Zealand            7.286  1.25018  1.31967          0.90837   
3           Mexico            7.187  1.02054  0.91451          0.81444   
4    United States            7.119  1.39451  1.24711          0.86179   
..             ...              ...      ...      ...              ...   
230        Lesotho            3.802  0.48900  1.16900          0.16800   
231        Burundi            3.775  0.04600  0.44700          0.38000   
232          Syria            3.462  0.61900  0.37800          0.44000   
233       Tanzania            3.231  0.47600  0.88500          0.49900   
234    Afghanistan            3.203  0.35000  0.51700          0.36100   

     freedom  corruption  generosity  prediccion  
0    0.66557     0.41978     0.29678    7